## Вариант 1.

#### Необходимо собрать информацию о вакансиях на вводимую должность с сайтов HH и Superjob. Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе:
- Наименование вакансии.
- Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
- Ссылку на саму вакансию.
- Сайт, откуда собрана вакансия.

Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
from time import sleep

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
URL_HH_PREFIX = 'https://hh.ru/search/vacancy?text='
URL_HH_SUFFIX = '&from=suggest_post&area=1'

In [3]:
def get_soups_from_url(url):
    response = requests.get(url, headers = {'User-Agent': USER_AGENT})
    soups = []
    if response.status_code == 200:
        soups.append(bs(response.content, 'html.parser'))
        while soups[-1].find_all('a', {'data-qa': 'pager-next'}):
            sleep(5)
            response = requests.get(url = url_hh + '&page=' + str(len(soups)), headers = {'User-Agent': USER_AGENT})
            if response.status_code == 200:
                soups.append(bs(response.content, 'html.parser'))
            else:
                print(f'Парсинг завершен досрочно. Удалось спарсить только страниц: {len(soups)}')
                break
        else:
            print(f'Парсинг успешно завершен. Удалось спарсить страниц: {len(soups)}')
    else:
        print(f'Парсинг не выполнен. Проверьте URL или другие параметры')
    return soups

In [4]:
def get_vacancies_from_soups_hh(url):
    vacancies = {}
    vacancy_id = 1
    for soup in soups:
        for vacancy in soup.find_all('div', {'class': 'serp-item'}):
            vacancy_name = vacancy.find('a', {'class': 'serp-item__title'}).text
            vacancy_company = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).text
            vacancy_salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
            salary_min, salary_max, salary_currency = '', '', ''
            if vacancy_salary:
                vacancy_salary = vacancy_salary.text
                salary_currency = vacancy_salary[vacancy_salary.rfind(' ')+1:].upper().replace('.','')
                vacancy_salary = vacancy_salary[:vacancy_salary.rfind(' ')]
                if vacancy_salary.find('–') > 0:
                    salary_min = vacancy_salary[:vacancy_salary.find('–')].strip().replace(' ','')
                    salary_max = vacancy_salary[vacancy_salary.find('–')+1:].strip().replace(' ','')
                elif vacancy_salary.find('от ') == 0:
                    salary_min = vacancy_salary[3:].strip().replace(' ','')
                elif vacancy_salary.find('до ') == 0:
                    salary_max = vacancy_salary[3:].strip().replace(' ','')
            vacancy_link = vacancy.find('a')['href']
            vacancy_source = 'HeadHunter'
            vacancies[vacancy_id] = {'vacancy_name': vacancy_name,
                                     'vacancy_company': vacancy_company,
                                     'salary_min': salary_min,
                                     'salary_max': salary_max,
                                     'salary_currency': salary_currency,
                                     'vacancy_link': vacancy_link,
                                     'vacancy_source': vacancy_source}
            vacancy_id += 1
    return vacancies

In [8]:
text = input('Введите профессию, должность или компанию: ')
url_hh = URL_HH_PREFIX + text.strip().replace(' ','+') + URL_HH_SUFFIX
soups = get_soups_from_url(url_hh)
get_vacancies_from_soups_hh(soups)

Введите профессию, должность или компанию: Data science
Парсинг успешно завершен. Удалось спарсить страниц: 9


{1: {'vacancy_name': 'Python-разработчик в Цифровые поверхности Салют (Сценарная разработка)',
  'vacancy_company': 'Сбер. IT',
  'salary_min': '',
  'salary_max': '',
  'salary_currency': '',
  'vacancy_link': 'https://lyubertsy.hh.ru/vacancy/75888105?from=vacancy_search_list&query=Data+science',
  'vacancy_source': 'HeadHunter'},
 2: {'vacancy_name': 'Product Manager (Data Science)',
  'vacancy_company': 'Luxemo Limited',
  'salary_min': '4000',
  'salary_max': '',
  'salary_currency': 'USD',
  'vacancy_link': 'https://lyubertsy.hh.ru/vacancy/77766765?from=vacancy_search_list&query=Data+science',
  'vacancy_source': 'HeadHunter'},
 3: {'vacancy_name': 'Стажер в Управление исследований и инноваций Сбера',
  'vacancy_company': 'Сбер для экспертов',
  'salary_min': '',
  'salary_max': '',
  'salary_currency': '',
  'vacancy_link': 'https://lyubertsy.hh.ru/vacancy/77732422?from=vacancy_search_list&query=Data+science',
  'vacancy_source': 'HeadHunter'},
 4: {'vacancy_name': 'Data Scientis